In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3
!pip3 install konlpy
!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"
import nltk
from nltk.util import ngrams
from nltk import word_tokenize
from nltk import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, MLEProbDist
nltk.download('punkt')
import numpy as np
import codecs
from tqdm import tqdm
import random
from konlpy.tag import Okt

In [ ]:
sentence = "나는 매일 아침 지하철을 탄다"

In [ ]:
# Tokenize text 
tokens = word_tokenize(sentence)
print(tokens)

['나는', '매일', '아침', '지하철을', '탄다']


In [ ]:
# Tokenize based on morpheme
tagger = Okt()

def tokenize(text):

  tokens = ['/'.join(t) for t in tagger.pos(text)]
  
  return tokens

tokens = tokenize(sentence)
print(tokens)

['나/Noun', '는/Josa', '매일/Noun', '아침/Noun', '지하철/Noun', '을/Josa', '탄다/Verb']


In [ ]:
# Convert tokens to N-gram
bigram = ngrams(tokens, 2)
trigram = ngrams(tokens, 3)

print("bigram: ")
for b in bigram:
  print(b)

print("\ntrigram: ")
for t in trigram:
  print(t)

bigram: 
('나/Noun', '는/Josa')
('는/Josa', '매일/Noun')
('매일/Noun', '아침/Noun')
('아침/Noun', '지하철/Noun')
('지하철/Noun', '을/Josa')
('을/Josa', '탄다/Verb')

trigram: 
('나/Noun', '는/Josa', '매일/Noun')
('는/Josa', '매일/Noun', '아침/Noun')
('매일/Noun', '아침/Noun', '지하철/Noun')
('아침/Noun', '지하철/Noun', '을/Josa')
('지하철/Noun', '을/Josa', '탄다/Verb')


In [ ]:
# Add tokens indicating the beginning and end of sentences through Padding
bigram = ngrams(tokens, 2, pad_left = True, pad_right = True, left_pad_symbol = "<s>", right_pad_symbol = "</s>")

print("bigrams with padding: ")
for b in bigram:
  print(b)

bigrams with padding: 
('<s>', '나/Noun')
('나/Noun', '는/Josa')
('는/Josa', '매일/Noun')
('매일/Noun', '아침/Noun')
('아침/Noun', '지하철/Noun')
('지하철/Noun', '을/Josa')
('을/Josa', '탄다/Verb')
('탄다/Verb', '</s>')


In [ ]:
# Download the 'Naver' movie review dataset
%%time
!wget -nc -q https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

In [ ]:
# Extract the text from the downloaded dataset
with codecs.open("ratings_train.txt", encoding='utf-8') as f:
  data = [line.split('\t') for line in f.read().splitlines()]
  data = data[1:150000]

print("데이터셋: ", data[:10])
docs = [row[1] for row in data]
print("\n텍스트 데이터: ", docs[:5])
print("\n문장 개수: ", len(docs))

데이터셋:  [['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'], ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'], ['10265843', '너무재밓었다그래서보는것을추천한다', '0'], ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'], ['6483659', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1'], ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'], ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0'], ['9443947', '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네', '0'], ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '1'], ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1']]

텍스트 데이터: ['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']

문장 개수:  149999


In [ ]:
# Add bigram of tokenized text to list
sentences = []

for d in tqdm(docs):
  tokens = tokenize(d)
  bigram = ngrams(tokens, 2, pad_left = True, pad_right = True, left_pad_symbol = "<s>", right_pad_symbol = "</s>")
  sentences += [t for t in bigram]

print(sentences[:10])

cfd = ConditionalFreqDist(sentences)
print(cfd["<s>"].most_common(5))

100%|██████████| 149999/149999 [05:51<00:00, 427.05it/s]


[('<s>', '아/Exclamation'), ('아/Exclamation', '더빙/Noun'), ('더빙/Noun', '../Punctuation'), ('../Punctuation', '진짜/Noun'), ('진짜/Noun', '짜증나네요/Adjective'), ('짜증나네요/Adjective', '목소리/Noun'), ('목소리/Noun', '</s>'), ('<s>', '흠/Noun'), ('흠/Noun', '.../Punctuation'), ('.../Punctuation', '포스터/Noun')]
[('정말/Noun', 2718), ('이/Noun', 2371), ('진짜/Noun', 2232), ('이/Determiner', 2115), ('영화/Noun', 2069)]


In [ ]:
# Function that returns the n most frequent words following a given token
def most_common(c, n, pos = None):
  
  if pos is None:
    return cfd[tokenize(c)[0]].most_common(n)
  else:
    return cfd["/".join([c, pos])].most_common(n)

print(most_common("나", 10))

[('는/Josa', 831), ('의/Josa', 339), ('만/Josa', 213), ('에게/Josa', 148), ('에겐/Josa', 84), ('랑/Josa', 81), ('한테/Josa', 50), ('참/Verb', 45), ('이/Determiner', 44), ('와도/Josa', 43)]


In [ ]:
# Estimating conditional probability based on the frequency of occurrence
cpd = ConditionalProbDist(cfd, MLEProbDist)
print(cpd[tokenize(".")[0]].prob("</s>"))

0.39102658679807606


In [ ]:
# Calculate the probability that token (w) appears together in a bigram after token (c)
def bigram_prob(c, w):

  context = tokenize(c)[0]
  word = tokenize(w)[0]
  
  return cpd[context].prob(word)

print(bigram_prob("이", "영화"))
print(bigram_prob("영화", "이"))

0.4010748656417948
0.0001576820735192668


In [ ]:
# Generate a sentence based on the probability
def generate_sentence(seed = None, debug = False):

  if seed is not None:
    import random
    random.seed(seed)

  c = "<s>"
  sentence = []
  
  while True:
    if c not in cpd:
      break
    
    w = cpd[c].generate()

    if w == "</s>":
      break

    word = w.split("/")[0]
    pos = w.split("/")[1]

    if c == "<s>":
      sentence.append(word.title())
    elif c in ["`", "\"","'","("]:
      sentence.append(word)
    elif word in ["'", ".", ",", ")", ":", ";", "?"]:
      sentence.append(word)
    elif pos in ["Josa", "Punctuation", "Suffix"]:
        sentence.append(word)
    elif w in ["임/Noun", "것/Noun", "는걸/Noun", "릴때/Noun",
                "되다/Verb", "이다/Verb", "하다/Verb", "이다/Adjective"]:
        sentence.append(word)
    else:
        sentence.append(" " + word)
    c = w

    if debug:
      print(w)

  return "".join(sentence)


print(generate_sentence(2))

generate_sentence(2, debug = True)

도리까지 본 영화 너무... 뭔가.. 최고네요. 하지만.. 눈물 낫다는건 또 영화에 들지 않는다. 근데 뭐야 어떻게 그렇게 착했던 윤재랑은 에바 그린 드레스 소리 듣는거임""" 에리 욧의 미모로 번돈으로 그렸으면 좋았을듯, 예전부터 제작비로 흥미진진. 거기 나오는 노래 완전 망쳐 버리네 미친 듯 최고의 얼굴을 뒤엎을 빌렸을 뿐 작품중이 보내는 클라스가 현재 한국에 가장 재미없었다.....................
도리/Noun
까지/Josa
본/Verb
영화/Noun
너무/Adverb
.../Punctuation
뭔가/Noun
../Punctuation
최고/Noun
네/Suffix
요/Josa
./Punctuation
하지만/Conjunction
../Punctuation
눈물/Noun
낫다는건/Verb
또/Noun
영화/Noun
에/Josa
들지/Verb
않는다/Verb
./Punctuation
근데/Adverb
뭐/Noun
야/Josa
어떻게/Adjective
그렇게/Adverb
착했던/Adjective
윤재/Noun
랑은/Josa
에바/Noun
그린/Noun
드레스/Noun
소리/Noun
듣는거/Verb
임/Noun
"""/Punctuation
에리/Noun
욧의/Noun
미모/Noun
로/Josa
번돈/Noun
으로/Josa
그렸으면/Verb
좋았을듯/Adjective
,/Punctuation
예전/Noun
부터/Josa
제작비/Noun
로/Josa
흥미진진/Noun
./Punctuation
거기/Noun
나오는/Verb
노래/Noun
완전/Noun
망쳐/Verb
버리네/Verb
미친/Adjective
듯/Noun
최고/Noun
의/Josa
얼굴/Noun
을/Josa
뒤엎/Noun
을/Josa
빌렸을/Verb
뿐/Noun
작품/Noun
중/Suffix
이/Josa
보내는/Verb
클라스/Noun
가/Josa
현재/Noun
한국/Noun
에/Josa
가장/Noun
재미없었다/Adjective
...................../Punctuatio

'도리까지 본 영화 너무... 뭔가.. 최고네요. 하지만.. 눈물 낫다는건 또 영화에 들지 않는다. 근데 뭐야 어떻게 그렇게 착했던 윤재랑은 에바 그린 드레스 소리 듣는거임""" 에리 욧의 미모로 번돈으로 그렸으면 좋았을듯, 예전부터 제작비로 흥미진진. 거기 나오는 노래 완전 망쳐 버리네 미친 듯 최고의 얼굴을 뒤엎을 빌렸을 뿐 작품중이 보내는 클라스가 현재 한국에 가장 재미없었다.....................'